
# Funciones de orden superior 

Una función de orden superior es aquella que puede tomar funciones como parámetro, o devolver una función como resultado, o ambas cosas.

#Funciones currificadas:

Las funciones de Haskell solo pueden tomar un parametro.
Que pasa cuando hacemos max 4 7?

In [ ]:
max 4 7

Para ver lo que en realidad sucede haremos lo siguiente

In [ ]:
let max4 = max 4
max4 7
max4 1

lo que sucede es que max 4 7 debe leerse como (max 4) 7
max recive un unico paramentro, en este caso 4 y nos da como resultado una funcion parcialmente aplicada, la cual no es mas que una variable para Haskell, esta nueva función recive un unico parametro, si este es mayor que cuatro lo devolvera, sino devolvera 4

Veamos ahora cuan simple es pasar una funcion por paramentro a Haskell, para esto implementaremos la funcion map'

In [ ]:
map' :: (a->b)->[a]->[b]
map' _ [] = []
map' f (x:xs) = (f x):map' f xs

In [ ]:
map' (+3) [4,1 ,6]
map' ('s':) ["olo","ol"]
map' ("ya "++) ["voy","fui"]
map' (++" gol") ["hice un","me hicieron un"]
map' (*2) [1,2,3]
map' (map' (2*)) [[1,2,3],[9, 0, 54]]

In [ ]:
filter' :: (a ->Bool)->[a]->[a]
filter' _ [] = []
filter' p (x:xs) 
  | p x = x:filter' p xs
  | otherwise = filter' p xs

In [ ]:
filter' (>5) [1,2,3]
filter' (>5) [11,4,5,6,7,1,8]

In [ ]:
miQuickSort :: (a -> a -> Bool) -> [a] -> [a]
miQuickSort _ [] = []
miQuickSort c (x:xs) = 
  let 
      menor = (`c` x)
      mayor = not.menor
      menores = miQuickSort c (filter' menor xs)
      mayores = miQuickSort c (filter' mayor xs)
  in menores ++ [x] ++ mayores

In [ ]:
miQuickSort (<) [6,1,78,12 -12, 2, 8 ,9 ,0]
miQuickSort (>) [6,1,78,12 -12, 2, 8 ,9 ,0]

normaMenor (x,y) (z,w) = sqrt (x*x+y*y) < sqrt (z*z+w*w)
porCoordX (x,_) (y,_) = x<y

let xs = [(1,2), (2,2),(0,0), (-2,-2)]
miQuickSort normaMenor xs
miQuickSort porCoordX xs

porLargo xs ys = length xs < length ys

miQuickSort porLargo ["Hola","Pedro","tengo habre","Eu!"]

miQuickSort (<) (map' length ["hola","pedro","tengo habre","lo"])


In [ ]:
miQuickSort2 :: (Ord b) => (a -> b) -> [a] -> [a]
miQuickSort2 _ [] = []
miQuickSort2 f (x:xs) = 
  let 
      menores = miQuickSort2 f [a | a <- xs, (f a) <= (f x)]
      mayores = miQuickSort2 f [a | a <- xs, (f a) > (f x)]
  in menores ++ [x] ++ mayores

miQuickSort2 length ["pipo","locura","purrete","amigo mio", "si"]

In [ ]:
norma (x,y) = sqrt (x*x + y*y)
let 
    l = [(1,2),(3,1),(5,0),(1,0),(-2,-1)]
miQuickSort2 norma l

# Funciones anónimas \ (lambdas)
suelen ser usadas cuando necesitamos una función una sola vez. Las creamos para pasarlas como parametros a funciones de orden superior.
La estructura de una funcion lambda es:

(\a b -> 2a/b)

Las funciones lambda son expresiones (devuelven un valor), por eso podemos pasarlas como parámetros a funciones de orden superior.

In [ ]:
filter (\(x,y)->(x + y)/2 >= 5) [(5,6), (5,4),(4,4),(8,1), (9,0),(9,2)]

In [ ]:
sumar3 x y z = x + y + z
:t sumar3
:t sumar3 1
:t sumar3 1 2
:t sumar3 1 2 3
:t 6

esto es porque:

In [ ]:
sumar3'= \x-> \y-> \z-> x + y + z
sumar3' 1 2 3

In [ ]:
flip' f = (\ x y -> f y x)
let noMayor = flip' (>)
3 `noMayor` 4
3 `noMayor` 1

# Pliegues (folds)


In [ ]:
Son patrones existentes para implementar iteraciones.

In [ ]:
elem' :: (Eq a)=> a -> [a] -> Bool
elem' y ys = foldl (\acc x-> if x==y then True else acc) False ys

In [ ]:
elem' 1 [1, 2, 3 ,4 ,5, 6]
elem' 5 [1 ,2 ,3 ,4 ,5 ,6]
elem' 9 [1 ,2 ,3, 4 ,5 ,6]
elem' 1 []

In [ ]:
foldl' :: (a->b->a)-> a->[b]->a
foldl' _ acc [] = acc
foldl' f acc (x:xs) = foldl' f (f acc x) xs

elem' :: (Eq a)=> a -> [a] -> Bool
elem' y ys = foldl' (\acc x-> (x==y) || acc) False ys

elem' 1 [1, 2, 3 ,4 ,5, 6]
elem' 5 [1 ,2 ,3 ,4 ,5 ,6]
elem' 9 [1 ,2 ,3, 4 ,5 ,6]
elem' 1 []

In [ ]:
map2 :: (a->b)->[a]->[b]
map2 f xs = foldr (\x acc -> f x :acc) [] xs


map2 (>3) (take 10 [1..])

In [ ]:
findKey :: (Eq k)=> k->[(k,v)]->v
findKey key xs = snd.head.filter(\(k,v)->key==k)$ xs

let agenda = [("pablo","12345656"),("juukua","98765432345678"),("mono", "0800-mono")]
findKey "pablo" agenda


findKey' :: (Eq k)=> k->[(k,v)]->Maybe v
findKey' _ [] = Nothing
findKey' key ((k,v):xs) = if key == k
                            then Just v
                            else findKey' key xs

findKey' "pablo" agenda
findKey' "rodolfo" agenda

findKey'' :: (Eq k)=> k->[(k,v)]->Maybe v
findKey'' key xs = foldr (\(k,v) acc ->if k==key then Just v else acc) Nothing xs

findKey'' "pablo" agenda
findKey'' "rodolfo" agenda

In [1]:
import qualified Data.Map as Map
Map.fromList agenda
Map.fromList$ ("pablo","123456780000"):agenda
Map.empty
Map.insert "movistar" "altos ladris" Map.empty

Not in scope: `agenda'

In [25]:
multiplicarPor x = (*) x 

:t multiplicarPor

Line 1: Eta reduce
Found:
multiplicarPor x = (*) x
Why not:
multiplicarPor = (*)

multiplicarPor :: forall a. Num a => a -> a -> a